In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 캐글 Walmart Recruiting - Store Sales Forecasting
## https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting
### 월마트 판매량(weekly sales) 예측

#### 1. 데이터 불러오기

In [ ]:
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
test.head()

In [ ]:
store = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
store.head()

In [ ]:
# store 데이터 전처리(Type 칼럼이 문자로 되어 있으므로 이를 숫자로 변환)

store['Type'] = store['Type'].replace({'A' : 0, 'B' : 1, 'C' : 2})
store.head()

In [ ]:
feature = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
feature.head()

In [ ]:
# feature 데이터 전처리(feature 데이터의 store칼럼을 그룹으로 묶은 뒤 각 그룹의 unemployment 평균 구하기)

ft_unemploy = feature.groupby('Store')['Unemployment'].mean()
ft_unemploy.head()

#### 2. 기존 train, test 데이터에 store 데이터와 ft_unemploy칼럼 합치기

In [ ]:
# train, test 데이터에 store 데이터를 Store 칼럼 기준으로 병합

train = pd.merge(train, store, on = 'Store', how = 'left')
test = pd.merge(test, store, on = 'Store', how = 'left')
display(train, test)

In [ ]:
# train, test 데이터에 ft_unemploy 칼럼을 Store 칼럼 기준으로 병합

train = pd.merge(train, ft_unemploy, on = 'Store', how = 'left')
test = pd.merge(test, ft_unemploy, on = 'Store', how = 'left')
display(train, test)

In [ ]:
train.info()

In [ ]:
test.info()

#### 3. Date 칼럼을 datetime 형식으로 바꾸고 yeay, month, day, week 칼럼 추가

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])

train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['week'] = train['Date'].dt.week     # 공휴일이 몇째주에 위치해 있는지 확인하기 위해 week칼럼 추가
train

In [ ]:
train['weeknum'] = np.ceil(train['day'] / 7)     # np.ceil = 올림, 각 달에 공휴일이 몇 번째 주에 있는지 확인
train.head()

In [ ]:
# train 데이터에서 만든 칼럼 그대로 test 데이터에도 추가

test['Date'] = pd.to_datetime(test['Date'])

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['week'] = test['Date'].dt.week
test['weeknum'] = np.ceil(test['day'] / 7)
test

#### 4. 추가한 칼럼들의 유용성 확인 - 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (16, 8))

# month 칼럼의 유용성 확인. Weekly_Sales: 정답 칼럼
sns.boxplot(train['month'], train['Weekly_Sales'], showfliers = False)     # showfilers = False 아웃라어이 제거

- 9월, 10월의 경우 최소값이 0이하로 나오는데 이는 반품이나 환불 때문에 발생

In [ ]:
# week칼럼의 유용성 확인

plt.figure(figsize = (16, 8))
sns.boxplot(train['week'], train['Weekly_Sales'], showfliers = False)

In [ ]:
# Type 칼럼의 유용성 확인

plt.figure(figsize = (16, 8))
sns.boxplot(train['Type'], train['Weekly_Sales'], showfliers = False)

In [ ]:
plt.figure(figsize = (16, 8))
sns.boxplot(train['day'], train['Weekly_Sales'], showfliers = False)

#### 5. 필요없는 칼럼 제외

In [ ]:
train2 = train.drop(['Date', 'Weekly_Sales'], axis = 1)
train2

In [ ]:
test2 = test.drop(['Date'], axis = 1)
test2

#### 6. 모델링(random forest regressor)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 모델 선언
rf = RandomForestRegressor(n_jobs = 4)     # n_jobs = 4는 cpu 4개 사용한다는 뜻. n_jobs = -1이라고 쓰면 cpu 최대로 사용

In [ ]:
%%time
rf.fit(train2, train['Weekly_Sales'])     # 모델 훈련

In [ ]:
result = rf.predict(test2)     # test2 결과 예측
result

In [ ]:
len(result)

#### 7. 제출양식 불러오기

In [ ]:
sub = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
sub

#### 8. 제출양식에 결과 포함

In [ ]:
sub['Weekly_Sales'] = result
sub

#### 9. 결과물 csv파일로 저장

In [ ]:
sub.to_csv('sub.csv', index = 0)     # index 불포함

#### 캐글 결과 제출 방법
- data -> output -> /kaggle/working -> ~.csv파일 다운로드
- data -> input -> (프로젝트 제목) -> more actions -> open in new tab
- late submission -> step1: csv파일 제출 -> step2: 간단한 설명 -> make submission